In [ ]:
!pip install -U langchain
# !pip install -U transformers accelerate

In [1]:
from torch import cuda, bfloat16
import transformers

token=""
model_id="epfl-llm/meditron-7b"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=token
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=token
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=token
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
stop_list = ["\n1"]
# stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x, add_special_tokens=False)['input_ids'] for x in stop_list]
stop_token_ids

[[29871, 13, 29896]]

In [8]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([29871,    13, 29896], device='cuda:0')]

In [9]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [10]:
from transformers import TextStreamer

generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    do_sample=True,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=256,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    streamer=TextStreamer(tokenizer)
)

In [ ]:
res = generate_text("Human: What are some common biomakers studied in cancer research?")
print(res[0]["generated_text"])

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory


llm = HuggingFacePipeline(pipeline=generate_text)

prompt = ChatPromptTemplate(
    messages=[
        # SystemMessagePromptTemplate.from_template(
        #     "You are a helpful, respectful and honest assistant." + \
        # "Always answer as helpfully as possible, while being safe." + \
        # "Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content." + \
        # "Please ensure that your responses are socially unbiased and positive in nature.\n\n" + \
        # "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct." + \
        # "If you don't know the answer to a question, please don't share false information."
        # ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("""
0:    Sentence:      Signed informed consent must be obtained prior to performing any specific pre-screening and screening procedure.
1:    biomarkers:    none
2:    Sentence:      Male or female >= 18 years of age at the time of informed consent
3:    biomarkers:    none
4:    Sentence:      Histologically or cytologically confirmed diagnosis of advanced/metastatic differentiated thyroid cancer
5:    biomarkers:    none
6:    Sentence:      BRAFV600E mutation positive tumor sample as per Novartis designated central laboratory result
7:    biomarkers:    BRAFV600E
8:    Sentence:      Concomitant RET Fusion Positive Thyroid cancer
9:    biomarkers:    RET Fusion Positive
10:   Sentence:      {question}
11:   biomarkers:    """),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

conversation({"question": "Participants must be ‚â• 18 years of age"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: 
0:    Sentence:      Signed informed consent must be obtained prior to performing any specific pre-screening and screening procedure.
1:    biomarkers:    none
2:    Sentence:      Male or female >= 18 years of age at the time of informed consent
3:    biomarkers:    none
4:    Sentence:      Histologically or cytologically confirmed diagnosis of advanced/metastatic differentiated thyroid cancer
5:    biomarkers:    none
6:    Sentence:      BRAFV600E mutation positive tumor sample as per Novartis designated central laboratory result
7:    biomarkers:    BRAFV600E
8:    Sentence:      Concomitant RET Fusion Positive Thyroid cancer
9:    biomarkers:    RET Fusion Positive
10:   Sentence:      Participants must be ‚â• 18 years of age
11:   biomarkers:    
of Human: 
0:    Sentence:      Signed informed consent must be obtained prior to performing any specific pre-screening and screening procedure.
1:    biomarkers:    no

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



12:   Sentence:      Participants must have a histologically or cytologically confirmed diagnosis of advanced/metastatic differentiated thyroid cancer
13:   biomarkers:    
14:   Sentence:      Participants must have a BRAF V600E mutation positive tumor sample as per Novartis designated central laboratory result
15:   biomarkers:    
16:   Sentence:      Participants must have a concomitant RET fusion positive thyroid cancer
17:   